In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, Dense, Activation, InputLayer
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [2]:
train = pd.read_csv("../../Data/Kaggle/StoreSales/processed_train_v2.csv")
test = pd.read_csv("../../Data/Kaggle/StoreSales/processed_test_v2.csv")

In [3]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

In [4]:
train.set_index('date', inplace=True)
test.set_index('date', inplace=True)

In [5]:
train.head()

,id,store_nbr,family,sales,onpromotion,typeholiday,dcoilwtico,city,state,typestores,cluster,day_of_week,day,month,year
date,,,,,,,,,,,,,,,
2013-01-01,0,1,0,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
2013-01-01,1,1,1,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
2013-01-01,2,1,2,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
2013-01-01,3,1,3,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
2013-01-01,4,1,4,0.0,0,3,93.14,18,12,3,13,2,1,1,2013


In [6]:
test.head()

,id,store_nbr,family,onpromotion,typeholiday,dcoilwtico,city,state,typestores,cluster,day_of_week,day,month,year
date,,,,,,,,,,,,,,
2017-08-16,3000888,1,0,0,4,46.8,18,12,3,13,3,16,8,2017
2017-08-16,3000889,1,1,0,4,46.8,18,12,3,13,3,16,8,2017
2017-08-16,3000890,1,2,2,4,46.8,18,12,3,13,3,16,8,2017
2017-08-16,3000891,1,3,20,4,46.8,18,12,3,13,3,16,8,2017
2017-08-16,3000892,1,4,0,4,46.8,18,12,3,13,3,16,8,2017


In [7]:
X = train.drop(['id','sales'], axis=1)
y = train['sales']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
time_steps = 10

In [11]:
X_train_reshaped = []
y_train_reshaped = []

In [12]:
for i in range(len(X_train_scaled) - time_steps):
    X_train_reshaped.append(X_train_scaled[i:i + time_steps])
    y_train_reshaped.append(y_train.values[i + time_steps])

In [13]:
X_train_reshaped = np.array(X_train_reshaped)
y_train_reshaped = np.array(y_train_reshaped)

In [14]:
X_test_reshaped = []
y_test_reshaped = []

In [15]:
for i in range(len(X_test_scaled) - time_steps):
    X_test_reshaped.append(X_test_scaled[i:i + time_steps])
    y_test_reshaped.append(y_test.values[i + time_steps])

In [16]:
X_test_reshaped = np.array(X_test_reshaped)
y_test_reshaped = np.array(y_test_reshaped)

In [17]:
model = Sequential()
model.add(InputLayer(input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(LSTM(64, return_sequences=True))
# model.add(Dense(64))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(30, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))

In [18]:
learning_rate = 1e-3

In [19]:
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

In [ ]:
model.fit(X_train_reshaped, y_train_reshaped, epochs=10, batch_size=1000, validation_split=0.2)

Epoch 1/10
1921/1921 [==============================] - 62s 32ms/step - loss: 1231381.1250 - val_loss: 1221776.5000
Epoch 2/10
1921/1921 [==============================] - 60s 31ms/step - loss: 1206923.6250 - val_loss: 1219939.0000
Epoch 3/10
1921/1921 [==============================] - 62s 32ms/step - loss: 1205683.8750 - val_loss: 1219066.2500
Epoch 4/10
1921/1921 [==============================] - 67s 35ms/step - loss: 1205190.7500 - val_loss: 1218853.8750
Epoch 5/10
1861/1921 [============================>.] - ETA: 31s - loss: 1206087.5000

In [ ]:
test_loss = model.evaluate(X_test_reshaped, y_test_reshaped)
print(f'Test Loss: {test_loss}')

In [ ]:
y_pred = model.predict(X_test_reshaped)

In [ ]:
model.save('M19.h5')

In [ ]:
y_test_reshaped.shape

In [ ]:
y_pred.shape

In [ ]:
y_pred_flat = y_pred.reshape(-1)
y_pred_flat = y_pred_flat[:len(y_test_reshaped_flat)]

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test_reshaped, y_pred_flat))
print("RMSE:", rmse)

In [ ]:
import matplotlib.pyplot as plt

# Assuming y_test_reshaped_flat and y_pred_flat are defined
# Adjust the length of y_pred_flat to match y_test_reshaped_flat

# Plotting actual values
plt.plot(y_test_reshaped_flat[:1000], label='Actual')

# Plotting predicted values
plt.plot(y_pred_flat[:1000], label='Predicted')

plt.title('Actual vs Predicted Values (First 1000)')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.show()